In [ ]:
import numpy as np
import pandas as pd
from numba import jit 
from numba.experimental import jitclass
from numba.typed import List

import source.riversectionNumba as sect
import source.s1driverflow as model

In [ ]:
import holoviews as hv
hv.extension('bokeh')

# flow model

## non-uniform flow model

In [ ]:
def nonuniform(channel, Qini):
    Qt = np.full(len(channel), Qini, dtype=np.float64)
    ib = float(1.0/400.0)
    
    H0 = channel[0].calH0ABS(Qini,ib)[0]
    
    Huni = model.NonUniformflow(channel, Qt, H0)
    
    Aini = []
    for i, hh in enumerate(Huni):
        Aini.append(channel[i].H2ABS(hh)[0])
    Aini = np.array(Aini)
    
    return Huni, Aini

## unsteady flow model  by collocated grid

In [ ]:
def unsteady(ch, chrr, Qini, kind=1, dt=float(10)):
    ib = float(1.0/400.0)
    Huni, Auni = nonuniform(ch, Qini)
    Q = np.full_like(Auni, Qini)
    A, H = Auni[::-1], Huni[::-1]
    
    for n in range(1, int(3600*10/dt)):
#     for n in range(1, 2):
        H0, A0, _, _ = chrr[-1].calH0ABS(Qini,ib)
        if kind == 1 : # collocated
            A, Q, H = model.UnSteadyflowCollocated(chrr, A, Q, H, A0, Qini, dt)
        elif kind == 2 : # staggered
            A, Q, H = model.UnSteadyflowStaggered(chrr, A, Q, H, A0, Qini, dt)
        
    return A, Q, H

# caclculation

In [ ]:
df = pd.read_csv('zb.csv', index_col=0)

In [ ]:
B = 50
manning = 0.03
ib = 1/400
dep = 5
A = dep * B
S = B+2*dep
Qconst = ib**0.5 * (A/S)**(2/3) / manning * A
Qconst

In [ ]:
Frnorm = Qconst/A/np.sqrt(float(9.8)*(A/S))
Frnorm

## dx = 200

In [ ]:
channel = List()
for zbs, Ls in zip(df.zb.values, df.L.values) :

    p = np.array([
            [0, zbs+10]
            ,[0, zbs]
            ,[B, zbs]
            ,[B, zbs+10]
        ])
    
    n = np.repeat(manning, (len(p) - 1))
    
    typed_ps = List()
    typed_ns = List()
    
    typed_ps.append(p)
    typed_ns.append(n)
    
    channel.append( sect.section(typed_ps, typed_ns, Ls) )

channelr = List(reversed(channel))

In [ ]:
zb = [c.zbmin() for c in channel]
L = [c.distance for c in channel]

In [ ]:
H, A = nonuniform(channel, Qconst)
Acol, Qcol, Hcol = unsteady(channel, channelr, Qconst, 1)
Astg, Qstg, Hstg = unsteady(channel, channelr, Qconst, 2)

In [ ]:
Fr = []
Frcol = []
Frstg = []
for i,c in enumerate(channel):
    ie, Alpha, _, Rc, _, A, _, _ = c.calIeAlphaBetaRcUsubABS(Qconst, H[i])
    Fr.append( Qconst/A/np.sqrt( float(9.8)*Rc/Alpha ) )
    ie, Alpha, _, Rc, _, A, _, _ = c.calIeAlphaBetaRcUsubABS(Qconst, Hcol[::-1][i])
    Frcol.append( Qconst/A/np.sqrt( float(9.8)*Rc/Alpha ) )
    ie, Alpha, _, Rc, _, A, _, _ = c.calIeAlphaBetaRcUsubABS(Qconst, Hstg[::-1][i])
    Frstg.append( Qconst/A/np.sqrt( float(9.8)*Rc/Alpha ) )

In [ ]:
g = hv.Curve((L,H), label='non-uniform:dx=200') \
* hv.Curve((L,Hcol[::-1]), label='unsteady_collocated:dx=200') \
* hv.Curve((L,Hstg[::-1]), label='unsteady_staggered:dx=200') \
* hv.Curve((L,df['zbmean'].values+5.0), label='normal-depth(on zbmean)').options(color='k',line_width=2,line_dash='dashed') \
* hv.Curve((L,df['zb'].values), label='zb').options(color='k') \
* hv.Curve((L,df['zbmean'].values), label='zbmean').options(color='gray')

go = g.options(width=600,height=450,show_grid=True, legend_position='top_left', ylabel='elevation')

In [ ]:
g2 = hv.Curve((L,Fr), vdims='Fr',label='non-uniform:dx=200') \
* hv.Curve((L,Frcol), vdims='Fr',label='unsteady_collocated:dx=200') \
* hv.Curve((L,Frstg), vdims='Fr',label='unsteady_staggered:dx=200') \
* hv.HLine(Frnorm).options(color='k',line_width=2,line_dash='dashed')

go2 = g2.options(width=600,height=200,show_grid=True, show_legend=False, ylabel='Froude number')

In [ ]:
gall = (go + go2).cols(1)

In [ ]:
hv.save(gall,'fig1.html')

## dx = 50

In [ ]:
zbt, Lt = df.zb.values, df.L.values
L2 = np.arange(Lt[0],Lt[-1]+1,50, dtype=float)
zb2 = np.interp(L2, Lt, zbt)

In [ ]:
channel = List()
for zbs, Ls in zip(zb2, L2) :

    p = np.array([
            [0, zbs+10]
            ,[0, zbs]
            ,[B, zbs]
            ,[B, zbs+10]
        ])
    
    n = np.repeat(manning, (len(p) - 1))
    
    typed_ps = List()
    typed_ns = List()
    
    typed_ps.append(p)
    typed_ns.append(n)
    
    channel.append( sect.section(typed_ps, typed_ns, Ls) )

channelr = List(reversed(channel))

In [ ]:
H2, A2 = nonuniform(channel, Qconst)
Acol2, Qcol2, Hcol2 = unsteady(channel, channelr, Qconst, 1, dt=float(5))
Astg2, Qstg2, Hstg2 = unsteady(channel, channelr, Qconst, 2, dt=float(2.5))

In [ ]:
from holoviews.plotting.util import process_cmap
cmap_list = process_cmap("glasbey_hv")

In [ ]:
g = hv.Curve((L,H), label='non-uniform:dx=200') \
* hv.Curve((L,Hcol[::-1]), label='unsteady_collocated:dx=200') \
* hv.Curve((L,Hstg[::-1]), label='unsteady_staggered:dx=200') \
* hv.Curve((L2,H2), label='non-uniform:dx=50').options(color=cmap_list[0],line_dash='dashed') \
* hv.Curve((L2,Hcol2[::-1]), label='unsteady_collocated:dx=50').options(color=cmap_list[1],line_dash='dashed') \
* hv.Curve((L2,Hstg2[::-1]), label='unsteady_staggered:dx=50').options(color=cmap_list[2],line_dash='dashed') \
* hv.Curve((L,df['zbmean'].values+5.0), label='normal-depth(on zbmean)').options(color='k',line_width=2,line_dash='dashed') \
* hv.Curve((L,df['zb'].values), label='zb').options(color='k') \
* hv.Curve((L,df['zbmean'].values), label='zbmean').options(color='gray')

go = g.options(width=600,height=550,show_grid=True, legend_position='top_left', ylabel='elevation')

In [ ]:
hv.save(go,'fig2.html')